In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [4]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [5]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [6]:
gen = Generator(z_dim = 106)
load_model(gen, "U0A0 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_1 = gen(to_gen).detach().numpy()
y_1 = np.zeros(35)

gen = Generator(z_dim = 106)
load_model(gen, "U0A1 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_2 = gen(to_gen).detach().numpy()
y_2 = np.ones(35)

gen = Generator(z_dim = 106)
load_model(gen, "U0A2 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_3 = gen(to_gen).detach().numpy()
y_3 = np.ones(35) + 1

gen = Generator(z_dim = 106)
load_model(gen, "U1A0 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_4 = gen(to_gen).detach().numpy()
y_4 = np.ones(35) + 2

gen = Generator(z_dim = 106)
load_model(gen, "U1A1 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_5 = gen(to_gen).detach().numpy()
y_5 = np.ones(35) + 3

gen = Generator(z_dim = 106)
load_model(gen, "U1A2 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_6 = gen(to_gen).detach().numpy()
y_6 = np.ones(35) + 4

gen = Generator(z_dim = 106)
load_model(gen, "U2A0 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_7 = gen(to_gen).detach().numpy()
y_7 = np.ones(35) + 5

gen = Generator(z_dim = 106)
load_model(gen, "U2A1 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_8 = gen(to_gen).detach().numpy()
y_8 = np.ones(35) + 6

gen = Generator(z_dim = 106)
load_model(gen, "U2A2 Excluded Group 1_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_9 = gen(to_gen).detach().numpy()
y_9 = np.ones(35) + 7

X_test = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
y_test = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

In [7]:
activities = [1, 3, 4]
users = [1, 3, 5]
X_train, y_train = start_data(activities, users, "Activity", sub_features, act_features)

In [8]:
model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.436426877975464, Final Batch Loss: 2.2291717529296875
Epoch 2, Loss: 4.4310455322265625, Final Batch Loss: 2.230130195617676
Epoch 3, Loss: 4.4253387451171875, Final Batch Loss: 2.2210352420806885
Epoch 4, Loss: 4.424116134643555, Final Batch Loss: 2.2234346866607666
Epoch 5, Loss: 4.4197587966918945, Final Batch Loss: 2.2062106132507324
Epoch 6, Loss: 4.418846607208252, Final Batch Loss: 2.2107017040252686
Epoch 7, Loss: 4.4090735912323, Final Batch Loss: 2.2020959854125977
Epoch 8, Loss: 4.403632402420044, Final Batch Loss: 2.1967828273773193
Epoch 9, Loss: 4.403399705886841, Final Batch Loss: 2.1970789432525635
Epoch 10, Loss: 4.403429985046387, Final Batch Loss: 2.209475040435791
Epoch 11, Loss: 4.398860931396484, Final Batch Loss: 2.2025015354156494
Epoch 12, Loss: 4.395235538482666, Final Batch Loss: 2.1900064945220947
Epoch 13, Loss: 4.38377046585083, Final Batch Loss: 2.1911420822143555
Epoch 14, Loss: 4.38370418548584, Final Batch Loss: 2.1901049613952637
Epoc

Epoch 115, Loss: 1.6839829683303833, Final Batch Loss: 0.8215690851211548
Epoch 116, Loss: 1.6216468811035156, Final Batch Loss: 0.8429588079452515
Epoch 117, Loss: 1.5856303572654724, Final Batch Loss: 0.7052487134933472
Epoch 118, Loss: 1.706580936908722, Final Batch Loss: 0.8682622909545898
Epoch 119, Loss: 1.6082261800765991, Final Batch Loss: 0.8483725190162659
Epoch 120, Loss: 1.5817093253135681, Final Batch Loss: 0.8023898601531982
Epoch 121, Loss: 1.5856998562812805, Final Batch Loss: 0.7947825789451599
Epoch 122, Loss: 1.589961290359497, Final Batch Loss: 0.8263174295425415
Epoch 123, Loss: 1.5764853358268738, Final Batch Loss: 0.7780173420906067
Epoch 124, Loss: 1.530397653579712, Final Batch Loss: 0.7950828075408936
Epoch 125, Loss: 1.6088129878044128, Final Batch Loss: 0.7982548475265503
Epoch 126, Loss: 1.6498457789421082, Final Batch Loss: 0.8949368000030518
Epoch 127, Loss: 1.5200533270835876, Final Batch Loss: 0.7794538736343384
Epoch 128, Loss: 1.6373355388641357, Fina

Epoch 233, Loss: 1.2172107696533203, Final Batch Loss: 0.6110031008720398
Epoch 234, Loss: 1.298873245716095, Final Batch Loss: 0.6825545430183411
Epoch 235, Loss: 1.2533661723136902, Final Batch Loss: 0.6417868733406067
Epoch 236, Loss: 1.2942447066307068, Final Batch Loss: 0.6749334931373596
Epoch 237, Loss: 1.2214624881744385, Final Batch Loss: 0.6052015423774719
Epoch 238, Loss: 1.225910484790802, Final Batch Loss: 0.6014999151229858
Epoch 239, Loss: 1.267816960811615, Final Batch Loss: 0.6609687805175781
Epoch 240, Loss: 1.2330355644226074, Final Batch Loss: 0.6585784554481506
Epoch 241, Loss: 1.2217450141906738, Final Batch Loss: 0.6165177822113037
Epoch 242, Loss: 1.241684913635254, Final Batch Loss: 0.5933823585510254
Epoch 243, Loss: 1.2872781157493591, Final Batch Loss: 0.648409366607666
Epoch 244, Loss: 1.1859389543533325, Final Batch Loss: 0.6204643249511719
Epoch 245, Loss: 1.2794899344444275, Final Batch Loss: 0.6715895533561707
Epoch 246, Loss: 1.2906911373138428, Final 

Epoch 364, Loss: 1.0133461952209473, Final Batch Loss: 0.5086928606033325
Epoch 365, Loss: 0.9728292226791382, Final Batch Loss: 0.4645129442214966
Epoch 366, Loss: 0.9169464111328125, Final Batch Loss: 0.4166707396507263
Epoch 367, Loss: 0.9173822999000549, Final Batch Loss: 0.4693338871002197
Epoch 368, Loss: 0.9402782618999481, Final Batch Loss: 0.5169180631637573
Epoch 369, Loss: 0.9567766785621643, Final Batch Loss: 0.46405017375946045
Epoch 370, Loss: 0.9765393137931824, Final Batch Loss: 0.5213350653648376
Epoch 371, Loss: 1.0019967555999756, Final Batch Loss: 0.5072609782218933
Epoch 372, Loss: 0.9726221561431885, Final Batch Loss: 0.4770532250404358
Epoch 373, Loss: 0.9182192087173462, Final Batch Loss: 0.4771227538585663
Epoch 374, Loss: 0.9195526540279388, Final Batch Loss: 0.4458175599575043
Epoch 375, Loss: 0.9450108110904694, Final Batch Loss: 0.5204097628593445
Epoch 376, Loss: 1.008950412273407, Final Batch Loss: 0.5268309712409973
Epoch 377, Loss: 0.960371196269989, Fi

Epoch 486, Loss: 0.7262003719806671, Final Batch Loss: 0.3460754156112671
Epoch 487, Loss: 0.6891772449016571, Final Batch Loss: 0.33585211634635925
Epoch 488, Loss: 0.6726702749729156, Final Batch Loss: 0.3321797847747803
Epoch 489, Loss: 0.7678123414516449, Final Batch Loss: 0.37074488401412964
Epoch 490, Loss: 0.7393369078636169, Final Batch Loss: 0.3806057274341583
Epoch 491, Loss: 0.7408308684825897, Final Batch Loss: 0.36471062898635864
Epoch 492, Loss: 0.7045098841190338, Final Batch Loss: 0.300109326839447
Epoch 493, Loss: 0.7320595681667328, Final Batch Loss: 0.3482154905796051
Epoch 494, Loss: 0.7008711993694305, Final Batch Loss: 0.35975417494773865
Epoch 495, Loss: 0.7314071953296661, Final Batch Loss: 0.32364925742149353
Epoch 496, Loss: 0.6775670647621155, Final Batch Loss: 0.3360159397125244
Epoch 497, Loss: 0.7218075692653656, Final Batch Loss: 0.34494733810424805
Epoch 498, Loss: 0.7550051510334015, Final Batch Loss: 0.40119025111198425
Epoch 499, Loss: 0.7038206160068

...

Epoch 4504, Loss: 0.28706932067871094, Final Batch Loss: 0.1465466469526291
Epoch 4505, Loss: 0.2727941647171974, Final Batch Loss: 0.11313840001821518
Epoch 4506, Loss: 0.3323659673333168, Final Batch Loss: 0.2118019014596939
Epoch 4507, Loss: 0.26818282902240753, Final Batch Loss: 0.13773228228092194
Epoch 4508, Loss: 0.32644154131412506, Final Batch Loss: 0.11905373632907867
Epoch 4509, Loss: 0.25116128474473953, Final Batch Loss: 0.11828992515802383
Epoch 4510, Loss: 0.28203389048576355, Final Batch Loss: 0.1512383073568344
Epoch 4511, Loss: 0.2777627110481262, Final Batch Loss: 0.13942687213420868
Epoch 4512, Loss: 0.3495994210243225, Final Batch Loss: 0.13935256004333496
Epoch 4513, Loss: 0.2949339598417282, Final Batch Loss: 0.16369298100471497
Epoch 4514, Loss: 0.2559630200266838, Final Batch Loss: 0.11128824204206467
Epoch 4515, Loss: 0.2716275602579117, Final Batch Loss: 0.1315038651227951
Epoch 4516, Loss: 0.2556762546300888, Final Batch Loss: 0.14445239305496216
Epoch 4517,

Epoch 4615, Loss: 0.25121647864580154, Final Batch Loss: 0.12802791595458984
Epoch 4616, Loss: 0.27686989307403564, Final Batch Loss: 0.14008952677249908
Epoch 4617, Loss: 0.28923071920871735, Final Batch Loss: 0.15209610760211945
Epoch 4618, Loss: 0.30320771038532257, Final Batch Loss: 0.125981405377388
Epoch 4619, Loss: 0.27440887689590454, Final Batch Loss: 0.11632546782493591
Epoch 4620, Loss: 0.24993164092302322, Final Batch Loss: 0.13599540293216705
Epoch 4621, Loss: 0.23498667031526566, Final Batch Loss: 0.1192501038312912
Epoch 4622, Loss: 0.27348238229751587, Final Batch Loss: 0.1269986629486084
Epoch 4623, Loss: 0.26217515021562576, Final Batch Loss: 0.12033679336309433
Epoch 4624, Loss: 0.2729981392621994, Final Batch Loss: 0.11935311555862427
Epoch 4625, Loss: 0.26947037875652313, Final Batch Loss: 0.14403414726257324
Epoch 4626, Loss: 0.3172641098499298, Final Batch Loss: 0.16636903584003448
Epoch 4627, Loss: 0.2878086417913437, Final Batch Loss: 0.13376668095588684
Epoch 

Epoch 4728, Loss: 0.2549392133951187, Final Batch Loss: 0.1282728612422943
Epoch 4729, Loss: 0.29637742042541504, Final Batch Loss: 0.14801180362701416
Epoch 4730, Loss: 0.27073366940021515, Final Batch Loss: 0.14872165024280548
Epoch 4731, Loss: 0.37040895223617554, Final Batch Loss: 0.1445566862821579
Epoch 4732, Loss: 0.23584553599357605, Final Batch Loss: 0.13487893342971802
Epoch 4733, Loss: 0.2775426506996155, Final Batch Loss: 0.1369563788175583
Epoch 4734, Loss: 0.23317991197109222, Final Batch Loss: 0.11853980273008347
Epoch 4735, Loss: 0.25910231471061707, Final Batch Loss: 0.12909206748008728
Epoch 4736, Loss: 0.23526304960250854, Final Batch Loss: 0.08210872113704681
Epoch 4737, Loss: 0.28430475294589996, Final Batch Loss: 0.13573673367500305
Epoch 4738, Loss: 0.2690410017967224, Final Batch Loss: 0.11736786365509033
Epoch 4739, Loss: 0.2899235114455223, Final Batch Loss: 0.17264145612716675
Epoch 4740, Loss: 0.2657451927661896, Final Batch Loss: 0.13559682667255402
Epoch 4

Epoch 4841, Loss: 0.2511315271258354, Final Batch Loss: 0.14425183832645416
Epoch 4842, Loss: 0.2904979735612869, Final Batch Loss: 0.14128130674362183
Epoch 4843, Loss: 0.21504785120487213, Final Batch Loss: 0.10574931651353836
Epoch 4844, Loss: 0.2798169404268265, Final Batch Loss: 0.12163394689559937
Epoch 4845, Loss: 0.24069586396217346, Final Batch Loss: 0.14535574615001678
Epoch 4846, Loss: 0.27466626465320587, Final Batch Loss: 0.12561510503292084
Epoch 4847, Loss: 0.2617490589618683, Final Batch Loss: 0.1342986524105072
Epoch 4848, Loss: 0.30897940695285797, Final Batch Loss: 0.16709810495376587
Epoch 4849, Loss: 0.2733786404132843, Final Batch Loss: 0.09802024066448212
Epoch 4850, Loss: 0.2757861241698265, Final Batch Loss: 0.10531390458345413
Epoch 4851, Loss: 0.22776900976896286, Final Batch Loss: 0.12410177290439606
Epoch 4852, Loss: 0.2619127631187439, Final Batch Loss: 0.13108234107494354
Epoch 4853, Loss: 0.270820677280426, Final Batch Loss: 0.14130571484565735
Epoch 485

Epoch 4963, Loss: 0.26286157220602036, Final Batch Loss: 0.14943668246269226
Epoch 4964, Loss: 0.25064025074243546, Final Batch Loss: 0.12036911398172379
Epoch 4965, Loss: 0.26538845896720886, Final Batch Loss: 0.14262928068637848
Epoch 4966, Loss: 0.2497604936361313, Final Batch Loss: 0.12439297139644623
Epoch 4967, Loss: 0.2573613375425339, Final Batch Loss: 0.12553168833255768
Epoch 4968, Loss: 0.24052844941616058, Final Batch Loss: 0.12840373814105988
Epoch 4969, Loss: 0.21852248907089233, Final Batch Loss: 0.11583758145570755
Epoch 4970, Loss: 0.29862524569034576, Final Batch Loss: 0.1751428097486496
Epoch 4971, Loss: 0.21840111911296844, Final Batch Loss: 0.08902603387832642
Epoch 4972, Loss: 0.27792317420244217, Final Batch Loss: 0.15859897434711456
Epoch 4973, Loss: 0.27024471014738083, Final Batch Loss: 0.16239576041698456
Epoch 4974, Loss: 0.26397348940372467, Final Batch Loss: 0.12262731790542603
Epoch 4975, Loss: 0.23206942528486252, Final Batch Loss: 0.11626560986042023
Ep

In [21]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 3))

[[29  1  0  0  0  0  5  0  0]
 [ 2 33  0  0  0  0  0  0  0]
 [33  0  2  0  0  0  0  0  0]
 [ 0  0  0 35  0  0  0  0  0]
 [ 0  0  0  0 35  0  0  0  0]
 [ 0  0  0  0  0 14  0  0 21]
 [ 0  0  0  0  0  0 35  0  0]
 [ 0 10  0  0  1  0 12 12  0]
 [ 0  0  0  0  0 24  0  0 11]]
              precision    recall  f1-score   support

         0.0      0.453     0.829     0.586        35
         1.0      0.750     0.943     0.835        35
         2.0      1.000     0.057     0.108        35
         3.0      1.000     1.000     1.000        35
         4.0      0.972     1.000     0.986        35
         5.0      0.368     0.400     0.384        35
         6.0      0.673     1.000     0.805        35
         7.0      1.000     0.343     0.511        35
         8.0      0.344     0.314     0.328        35

    accuracy                          0.654       315
   macro avg      0.729     0.654     0.616       315
weighted avg      0.729     0.654     0.616       315



In [23]:
# GET RID OF 2, 5, 8 for JUST DYNAMIC
(0.586 + 0.835 + 1 + 0.986 + 0.805 + 0.511) / 6

0.7871666666666667

In [24]:
(0.586+0.835+0.108+1+0.986+0.384+0.805+0.511+0.328)/9

0.6158888888888889